Clone the Github Repository and mount the Google Drive


Also enable GPU in Edit--> Notebook setting--> Hardware accelerator --> GPU

In [0]:
%%shell
git clone https://github.com/ansarisam/darknet.git


Cloning into 'darknet'...
remote: Enumerating objects: 5912, done.
remote: Total 5912 (delta 0), reused 0 (delta 0), pack-reused 5912
Receiving objects: 100% (5912/5912), 6.17 MiB | 4.03 MiB/s, done.
Resolving deltas: 100% (3925/3925), done.


In [0]:
%%shell
#rm -r /content/darknet

Download the Make file in darknet folder and replace gpu=1 and opencv=1 and upload back to same location in darknet folder.  

In [0]:
%%shell
cd darknet/
make 

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:957:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -Wall -Wno-unused-result 

Download the pre-trained weights to initialize the training

In [0]:
%%shell
cd /content/pretrained
wget https://pjreddie.com/media/files/darknet53.conv.74

/bin/bash: line 0: cd: /content/pretrained: No such file or directory
--2020-04-11 23:30:43--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  8.46MB/s    in 33s     

2020-04-11 23:31:17 (4.69 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



Copy the train and test images from drive to colab environment to the exact location specified below

In [0]:
!cp -r /content/drive/My\ Drive/Capstone/train_images/*.jpg /content/data/train_images

In [0]:
!cp -r /content/drive/My\ Drive/Capstone/test_images/*.jpg /content/data/test_images

In [0]:
#%cd /content

/content


In [0]:
%cd /content/drive/'My Drive'/Capstone/computer_vision/models/research

/content/drive/My Drive/Capstone/computer_vision/models/research


In [0]:
!pwd

/content/drive/My Drive/Capstone/computer_vision/models/research


In [0]:
import io
import os
from lxml import etree
from object_detection.utils import dataset_util
import tensorflow as tf

In [0]:
tf.test.gpu_device_name()

'/device:GPU:0'

Create train.txt and test.txt files which has absolute location to images in colab environment 

In [0]:
#train
train = open("/content/data/train.txt",'w')
for file in os.listdir("/content/drive/My Drive/Capstone/train_images"):
  train.write(os.path.join("/content/data/images/",file))
  train.write("\n")
train.close()

In [0]:
#test
test = open("/content/data/test.txt",'w')
for file in os.listdir("/content/drive/My Drive/Capstone/test_images"):
  test.write(os.path.join("/content/data/images/",file))
  test.write("\n")
test.close()

In [0]:
#!cp /content/data/train.txt /content/drive/My\ Drive/Capstone/

In [0]:
#!cp /content/data/test.txt /content/drive/My\ Drive/Capstone/

Function to convert XML to txt format

In [0]:
def xml_to_csv(annotations,def_class,in_path,out_path):
  count = 0 
  for idx,annot in enumerate(annotations):
    if annot.endswith('.xml'):
      path = os.path.join(in_path, annot)
      with tf.io.gfile.GFile(path, 'r') as fid:
        xml_str = fid.read()
      xml = etree.fromstring(xml_str)
      groundtruth_data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']
      h = int(groundtruth_data['size']['height'])  
      w = int(groundtruth_data['size']['width'])
      if '.jpg' in groundtruth_data['filename']:
        filename = groundtruth_data['filename']
        filename = filename.replace('.jpg','')
        filename = filename+'.txt'
      else:
        filename = groundtruth_data['filename']
        filename = filename+'.txt'
      num_annotations_skipped = 0
      write_file = open(out_path+filename,'w')
      #write_file.write("Class,x_center,y_centre,Width,Height")
      #write_file.write("\n")
      for objects in groundtruth_data['object']:
        defect = def_class[objects['name']]
        xmi = int(objects['bndbox']['xmin'])
        ymi = int(objects['bndbox']['ymin'])
        xma = int(objects['bndbox']['xmax'])
        yma = int(objects['bndbox']['ymax'])
        if xma <= 0 or yma <= 0:
          num_annotations_skipped += 1
          continue
        if xma > w or yma > h:
          num_annotations_skipped += 1
          continue
        x = float(((xmi+xma)/2) - 1)
        y = float(((ymi+yma)/2) - 1)
        xo = float(x/w)
        yo = float(y/h)
        ho = float((yma-ymi)/h)
        wo = float((xma-xmi)/w)
        write_file.write("%d %f %f %f %f"%(defect,xo,yo,wo,ho))
        write_file.write("\n")
      count += 1
      print("Total files written = ",count)
      print("Total annotations skipped = ",num_annotations_skipped)
      write_file.close()


In [0]:
defect = open('/content/drive/My Drive/Capstone/class.data','r')
defect_cls = defect.read()
defect_cls = defect_cls.split('\n')
defect_cls

['rolled-in_scale',
 'patches',
 'crazing',
 'pitted_surface',
 'inclusion',
 'scratches']

In [0]:
dict_class = {defect_cls[i]:i for i in range(0,len(defect_cls))}
dict_class

{'crazing': 2,
 'inclusion': 4,
 'patches': 1,
 'pitted_surface': 3,
 'rolled-in_scale': 0,
 'scratches': 5}

In [0]:
dict_class.values()

dict_values([0, 1, 2, 3, 4, 5])

Converting train xml files to txt

In [0]:
train_path = '/content/drive/My Drive/Capstone/train_annotations/'
train_out = '/content/data/annotations/train/'
train_annot = os.listdir(train_path)
xml_to_csv(train_annot,dict_class,train_path,train_out)

/content/drive/My Drive/Capstone/computer_vision/models/research/object_detection/utils/dataset_util.py:79: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:


Total files written =  1
Total annotations skipped =  0
Total files written =  2
Total annotations skipped =  0
Total files written =  3
Total annotations skipped =  0
Total files written =  4
Total annotations skipped =  0
Total files written =  5
Total annotations skipped =  0
Total files written =  6
Total annotations skipped =  0
Total files written =  7
Total annotations skipped =  0
Total files written =  8
Total annotations skipped =  0
Total files written =  9
Total annotations skipped =  0
Total files written =  10
Total annotations skipped =  0
Total files written =  11
Total annotations skipped =  0
Total files written =  12
Total annotations skipped =  0
Total files written =  13
Total annotations skipped =  0
Total files written =  14
Total annotations skipped =  0
Total files written =  15
Total annotations skipped =  0
Total files written =  16
Total annotations skipped =  0
Total files written =  17
Total annotations skipped =  0
Total files written =  18
Total annotati

Converting test xml files to txt

In [0]:
test_path = '/content/drive/My Drive/Capstone/test_annotations/'
test_out = '/content/data/annotations/test/'
test_annot = os.listdir(test_path)
xml_to_csv(test_annot,dict_class,test_path,test_out)

/content/drive/My Drive/Capstone/computer_vision/models/research/object_detection/utils/dataset_util.py:79: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:


Total files written =  1
Total annotations skipped =  0
Total files written =  2
Total annotations skipped =  0
Total files written =  3
Total annotations skipped =  0
Total files written =  4
Total annotations skipped =  0
Total files written =  5
Total annotations skipped =  0
Total files written =  6
Total annotations skipped =  0
Total files written =  7
Total annotations skipped =  0
Total files written =  8
Total annotations skipped =  0
Total files written =  9
Total annotations skipped =  0
Total files written =  10
Total annotations skipped =  0
Total files written =  11
Total annotations skipped =  0
Total files written =  12
Total annotations skipped =  0
Total files written =  13
Total annotations skipped =  0
Total files written =  14
Total annotations skipped =  0
Total files written =  15
Total annotations skipped =  0
Total files written =  16
Total annotations skipped =  0
Total files written =  17
Total annotations skipped =  0
Total files written =  18
Total annotati

Copy all the txt files to location in drive to avoid creating txt files every time the runtime gets disconnected

In [0]:
%%shell
#cp -r  /content/data/annotations/train/*.txt /content/drive/My\ Drive/Capstone/annotations/train/

In [0]:
%%shell
#cp /content/data/annotations/test/*.txt /content/drive/My\ Drive/Capstone/annotations/test/

NOTE: Creating the txt files and copying them to drive has to be done only once, i.e when you run this code for first time. From the second time we can simply copy the txt files from drive to colab as shown below 

Copying the txt files from drive to colab environment to specific location below

In [0]:
%%shell
cp -r /content/drive/My\ Drive/Capstone/annotations/train/*.txt /content/data/annotations/train/

In [0]:
%%shell
cp /content/drive/My\ Drive/Capstone/annotations/test/*.txt /content/data/annotations/test/

Create a labels folder in data and copy att the txt files of train and test into it

In [0]:
%%shell
cp /content/data/annotations/train/*.txt /content/data/labels

In [0]:
%%shell
cp /content/data/annotations/test/*.txt /content/data/labels

Create images folder inside data and copy all the images both train and test into that folder

In [0]:
%%shell
cp /content/data/train_images/*.jpg /content/data/images

In [0]:
%%shell
cp /content/data/test_images/*.jpg /content/data/images

In [0]:
%cd /content
!pwd

/content
/content


All train.txt, text.txt, class.data needs to be unix formatted files so we need to convert them from dos2unix as shown below

In [0]:
%%shell
sudo apt install dos2unix

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 1s (588 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package dos2unix.
(Reading database ... 144568 files and directories curren

Linking backupfolder in Drive to colab environment

In [0]:
!ln -s /content/drive/My\ Drive/Capstone/backup /content/yolov3_folder

In [0]:
!ls /content/yolov3_folder

yolov3_anomaly_network.backup	      yolov3-voc_two_10000.weights
yolov3_anomaly_network_final.weights  yolov3-voc_two_11000.weights
yolov3-voc_10000.weights	      yolov3-voc_two_12000.weights
yolov3-voc_11000.weights	      yolov3-voc_two_7000.weights
yolov3-voc_9000.weights		      yolov3-voc_two_8000.weights
yolov3-voc.backup		      yolov3-voc_two_9000.weights
yolov3-voc_made.backup		      yolov3-voc_two.backup
yolov3-voc_made_final.weights	      yolov3-voc_two_final.weights


Converting required files into dos2unix

In [0]:
%%shell
dos2unix /content/input.cfg
dos2unix /content/data/class.data

dos2unix: converting file /content/input.cfg to Unix format...
dos2unix: converting file /content/data/class.data to Unix format...


Training the Model

In [0]:
%%shell 
cd /content/darknet
./darknet detector train /content/input.cfg /content/yolov3-voc.cfg /content/yolov3_folder/yolov3-voc.backup 

Streaming output truncated to the last 5000 lines.
Region 82 Avg IOU: 0.816326, Class: 0.944387, Obj: 0.874410, No Obj: 0.005440, .5R: 1.000000, .75R: 0.750000,  count: 12
Region 94 Avg IOU: 0.891724, Class: 0.999779, Obj: 0.738340, No Obj: 0.000094, .5R: 1.000000, .75R: 1.000000,  count: 1
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000000, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.803135, Class: 0.998311, Obj: 0.872167, No Obj: 0.004336, .5R: 1.000000, .75R: 0.700000,  count: 10
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000039, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000000, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.872675, Class: 0.999282, Obj: 0.803382, No Obj: 0.002091, .5R: 1.000000, .75R: 1.000000,  count: 5
Region 94 Avg IOU: 0.771961, Class: 0.999957, Obj: 0.959177, No Obj: 0.000270, .5R: 1.000000, .75R: 0.500000,  count: 2
Region 106 Avg IOU: -nan, Class: -nan, Obj: -

Testing on Single image

In [0]:
%%shell
cd /content/darknet
./darknet detector test /content/input.cfg /content/yolov3-voc.cfg /content/yolov3_folder/yolov3-voc_18000.weights /content/data/test_images/crazing_204.jpg

 CUDA-version: 10010 (10010), GPU count: 1  
 OpenCV version: 3.2.0
 compute_capability = 600, cudnn_half = 0 
net.optimized_memory = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 1

CalledProcessError: ignored

In [0]:
#!rm -rf /content/darknet 

In [0]:
#!rm -r /content/output

Testing on all the test images 

We need to upload specific python code into darknet folder for generating predictions in json format

In [0]:
%%shell
cd /content/darknet/
python Text_worked.py

Testing 0 image crazing_288.jpg,
Image 0 written to output folder
Testing 1 image rolled-in_scale_224.jpg,
Image 1 written to output folder
Testing 2 image patches_224.jpg,
Image 2 written to output folder
Testing 3 image crazing_214.jpg,
Image 3 written to output folder
Testing 4 image inclusion_215.jpg,
Image 4 written to output folder
Testing 5 image rolled-in_scale_237.jpg,
Image 5 written to output folder
Testing 6 image inclusion_276.jpg,
Image 6 written to output folder
Testing 7 image scratches_290.jpg,
Image 7 written to output folder
Testing 8 image rolled-in_scale_274.jpg,
Image 8 written to output folder
Testing 9 image scratches_264.jpg,
Image 9 written to output folder
Testing 10 image pitted_surface_237.jpg,
Image 10 written to output folder
Testing 11 image pitted_surface_217.jpg,
Image 11 written to output folder
Testing 12 image scratches_243.jpg,
Image 12 written to output folder
Testing 13 image pitted_surface_242.jpg,
Image 13 written to output folder
Testing 14 im

CalledProcessError: ignored

Copying all the outputs to Drive

In [0]:
%%shell
cp  /content/output/*.json /content/drive/My\ Drive/Capstone/predictions/

In [0]:
%%shell
cd /content/darknet/
python Text_rough.py

0 crazing_275.jpg
1 inclusion_250.jpg
2 pitted_surface_243.jpg
3 pitted_surface_227.jpg
4 scratches_278.jpg
5 crazing_260.jpg
6 patches_214.jpg
7 crazing_266.jpg
8 pitted_surface_250.jpg
9 pitted_surface_225.jpg
10 patches_259.jpg
11 crazing_210.jpg
12 crazing_241.jpg
13 patches_289.jpg
14 rolled-in_scale_218.jpg
15 patches_247.jpg
16 patches_266.jpg
17 scratches_242.jpg
18 inclusion_291.jpg
19 patches_250.jpg
20 crazing_256.jpg
21 scratches_279.jpg
22 inclusion_240.jpg
23 inclusion_275.jpg
24 inclusion_271.jpg
25 rolled-in_scale_222.jpg
26 crazing_277.jpg
27 inclusion_234.jpg
28 scratches_201.jpg
29 pitted_surface_272.jpg
30 patches_221.jpg
31 patches_286.jpg
32 inclusion_262.jpg
33 inclusion_257.jpg
34 inclusion_218.jpg
35 patches_217.jpg
36 pitted_surface_226.jpg
37 patches_248.jpg
38 scratches_258.jpg
39 pitted_surface_292.jpg
40 inclusion_254.jpg
41 inclusion_227.jpg
42 pitted_surface_286.jpg
43 crazing_248.jpg
44 rolled-in_scale_226.jpg
45 rolled-in_scale_230.jpg
46 patches_226.j

In [0]:
#%%shell
#rm -r /content/data/test_images/.ipynb_checkpoints

Bounding Box predictions with images

In [0]:
import cv2,os,json
from google.colab.patches import cv2_imshow

In [0]:
color=(0,0,255)
thickness=1
for files in os.listdir('/content/drive/My Drive/Capstone/predictions/'):
  with open("/content/drive/My Drive/Capstone/predictions/"+files, 'r') as f:
    test_json = json.load(f)
  if len(test_json) == 0:
    continue
  else:
    print(test_json[0]['image'])
    path=test_json[0]['image']
    img=cv2.imread(path)
    for i in range(0,len(test_json)):
      a=test_json[i]['predictions']['bbox']
      start=(int(a[0]),int(a[1]))
      end=(int(a[2]),int(a[3]))
      x=int(a[0])
      if int(a[1]) > 5:
        y=int(a[1])-1
      else:
        y=int(a[1])+13
      predictedimg=cv2.rectangle(img,start,end,color,thickness)
      predictedimg=cv2.putText(predictedimg,test_json[i]['predictions']['object']+':'+test_json[i]['predictions']['score'],(x,y),cv2.FONT_HERSHEY_PLAIN,1,color,1,cv2.LINE_AA)
    cv2_imshow(predictedimg)

In [0]:
color=(255,0,0)
thickness=1
test_path = '/content/drive/My Drive/Capstone/test_annotations/'
#test_out = '/content/data/annotations/test/'
annotations = os.listdir(test_path)
for idx,annot in enumerate(annotations):
    if annot.endswith('.xml'):
      path = os.path.join(test_path, annot)
      with tf.io.gfile.GFile(path, 'r') as fid:
        xml_str = fid.read()
      xml = etree.fromstring(xml_str)
      groundtruth_data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']
      annot = annot.replace('.xml','.jpg')
      path = '/content/data/test_images/'+annot
      print(path)
      img=cv2.imread(path)
      for objects in groundtruth_data['object']:
        defect = objects['name']
        xmi = int(objects['bndbox']['xmin'])
        ymi = int(objects['bndbox']['ymin'])
        xma = int(objects['bndbox']['xmax'])
        yma = int(objects['bndbox']['ymax'])
        start=(xmi,ymi)
        end=(xma,yma)
        x=xmi
        if ymi > 5:
          y=ymi-1
        else:
          y=ymi+13
        predictedimg=cv2.rectangle(img,start,end,color,thickness)
        predictedimg=cv2.putText(predictedimg,defect,(x,y),cv2.FONT_HERSHEY_PLAIN,1,color,1,cv2.LINE_AA)
      cv2_imshow(predictedimg)
      

For measuring the performance of the model we need to delete previous darknet directory and clone alexAB darknet repository

In [0]:
!rm -r /content/darknet

In [0]:
%%shell
git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 13175 (delta 2), reused 5 (delta 2), pack-reused 13166
Receiving objects: 100% (13175/13175), 11.93 MiB | 24.57 MiB/s, done.
Resolving deltas: 100% (9006/9006), done.


Make similar changes in make file and upload it back

In [0]:
%%shell
cd darknet/
make 

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variable ‘it_tb_res’ [-Wunused-variable]
     int it_tb_res = cv::c

This repositry requires all images and lables to be in images directory so copy all labels to images directory

In [0]:
!cp /content/data/labels/*.txt /content/data/images

Claculating mAP, Precision, Recall and F1-score of the model

In [0]:
%%shell
cd /content/darknet
./darknet detector map /content/input.cfg /content/yolov3-voc.cfg /content/drive/My\ Drive/Capstone/yolo_steel_defect_final_weights/yolov3-voc_12000.weights

 CUDA-version: 10010 (10010), GPU count: 1  
 OpenCV version: 3.2.0
 compute_capability = 610, cudnn_half = 0 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 

In [0]:
%%shell
cd /content/darknet
./darknet detector calc_anchors /content/input.cfg -num_of_clusters 9 -width 416 -height 416

 CUDA-version: 10010 (10010), GPU count: 1  
 OpenCV version: 3.2.0

 num_of_clusters = 9, width = 416, height = 416 
 read labels from 1260 images 
 loaded 	 image: 1260 	 box: 2901
 all loaded. 

 calculating k-means++ ...

 iterations = 29 


counters_per_class = 439, 585, 459, 300, 760, 358

 avg IoU = 68.26 % 

Saving anchors to the file: anchors.txt 
anchors =  42, 95, 110, 94,  79,187,  46,353, 277,100, 166,179, 122,356, 315,188, 330,391


CalledProcessError: ignored